In [1]:
import cv2
import transformer
import torch
from torchvision import transforms, datasets

In [2]:
STYLE_MODEL_PATH = "transforms/udnie.pth"
WIDTH = 480
HEIGHT = 360

In [3]:
def imgtotensor(img):
    # Method to convert Image to Tensor
    if (True == True):
        itot_t = transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.mul(255))
        ]) 

    # Convert image to tensor
    tensor = itot_t(img)
    # Add the batch_size dimension
    tensor = tensor.unsqueeze(dim=0)
    return tensor


def tensortoimg(tensor):
    # Method to convert Tensor to Image

    # Remove the batch_size dimension
    tensor = tensor.squeeze()
    img = tensor.cpu().numpy()
    # Transpose from [C, H, W] -> [H, W, C]
    img = img.transpose(1, 2, 0)
    return img

In [4]:
def webcam(style_transform_path, width=1280, height=720):

    # Device
    device = ("cuda" if torch.cuda.is_available() else "cpu")

    # Load Transformer Network
    print("Loading Transformer Network")
    net = transformer.TransformerNetwork()
    net.load_state_dict(torch.load(STYLE_MODEL_PATH))
    net = net.to(device)
    print("Done Loading Transformer Network")

    # Set webcam settings
    cam = cv2.VideoCapture(0)
    cam.set(3, width)
    cam.set(4, height)
    fps = cam.get(cv2.CAP_PROP_FPS)

    # Main loop
    with torch.no_grad():
        while True:
            # Get webcam input
            ret_val, img = cam.read()

            # Mirror 
            img = cv2.flip(img, 1)

            # Free-up unneeded cuda memory
            torch.cuda.empty_cache()
            
            # Generate image
            content_tensor = imgtotensor(img).to(device)
            generated_tensor = net(content_tensor)
            generated_image = tensortoimg(generated_tensor.detach())
            generated_image = generated_image / 255
            
#             print("Fps: ",fps)

            # Show webcam
            cv2.imshow('Demo webcam', generated_image)
            if cv2.waitKey(1) == 27:
                torch.cuda.empty_cache()
                break  # esc to quit
            
    # Free-up memory
    cam.release()
    cv2.destroyAllWindows()

In [6]:
webcam(STYLE_MODEL_PATH, WIDTH, HEIGHT)

Loading Transformer Network
Done Loading Transformer Network
